# Spike Locator

## Written by Michael George

Iterate through folders, looking for possible spikes in GPS files

In [1]:
import os
import sys

import traceback

readersPath = os.path.join('.', 'readers')
if readersPath not in sys.path:
    sys.path.extend([readersPath])

from generic_reader import getReader

## Main Function

In [2]:
def findSpikes():
    '''Iterate through session archive testing each GPS file'''

    rootDir = os.path.join(projdir, 'sessions')

    errors = {}
    spikes = []

    for root, subDirs, files in os.walk(rootDir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext and ext in ('.gpx', '.oao', '.sbn', '.sbp', '.ubx'):
                filePath = os.path.join(root, file)
                reader = getReader(filePath)
                try:
                    # Some legacy ESP-GPS files contain bad checksums
                    if ext == '.ubx':
                        reader.load(ignoreChecksums=True)
                    else:
                        reader.load()
                    
                    # Ignore GT-11 files
                    if not ('firmware' in reader.header and reader.header['firmware'].startswith('V1.62')):
                        if 'sog' in reader.data:
                            maxSpeed = reader.data['sog'].max()
                            if maxSpeed > 25:
                                print('S', end='')
                                #print('\n{} m/s in {}'.format(maxSpeed, filePath.replace(projdir + '/', '')))
                                spikes.append(filePath.replace(projdir + '/', ''))
                            else:
                                print('.', end='')
                    else:
                        print('.', end='')
                except:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

    if len(spikes) > 0:
        print(os.linesep * 2 + 'Spikes:')
        for spike in spikes:
            print(spike)

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], ".."))

    findSpikes()
    
    print(os.linesep + 'All done!')

..SS.....SS........................................................................................S..................................................S.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................